<a href="https://colab.research.google.com/github/BlackLutos/Neural-Networks/blob/main/ml2022_hw2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## HW2

In [5]:
! pip install timm

     |████████████████████████████████| 431 kB 5.0 MB/s 


In [6]:
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import timm

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [8]:
# see more data augmentation https://pytorch.org/vision/stable/transforms.html
# mean = (0.5071, 0.4867, 0.4408)
# std = (0.2675, 0.2565, 0.2761)
mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)

train_transform = transforms.Compose(
    [#transforms.RandomHorizontalFlip(p=0.5),
     transforms.ToTensor(),
     transforms.Normalize(mean, std)]) # calculte yourself

test_transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize(mean, std)]) # calculte yourself

# batch_size = 32
batch_size = 200
num_classes = 100    # check

trainset = torchvision.datasets.CIFAR100(root='./data', train=True,
                                        download=True, transform=train_transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR100(root='./data', train=False,
                                       download=True, transform=test_transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)


  0%|          | 0/169001437 [00:00<?, ?it/s]

Extracting ./data/cifar-100-python.tar.gz to ./data
Files already downloaded and verified


In [12]:
testset

Dataset CIFAR100
    Number of datapoints: 10000
    Root location: ./data
    Split: Test
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.5071, 0.4867, 0.4408), std=(0.2675, 0.2565, 0.2761))
           )

In [5]:
class Toy_CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


In [9]:
from torch.nn.modules.activation import LogSoftmax
# pick one

# 1. model defined by yourself
# model = Toy_CNN()        
   
# 2. off-the-shelf model
# see https://pytorch.org/vision/stable/models.html
# nn.Linear https://pytorch.org/docs/stable/generated/torch.nn.Linear.html#torch.nn.Linear
model = models.resnet50(pretrained=True)
# Transfor Larning
for param in model.parameters():
    print(param.names)
    param.requires_grad = False

fc_inputs = model.fc.in_features
model.fc = torch.nn.Sequential(
    torch.nn.Linear(fc_inputs,num_classes),
    torch.nn.LogSoftmax(dim=1)
) 
#model.fc = torch.nn.Linear(2048, num_classes)
# model = timm.create_model('xception', pretrained=True, num_classes=num_classes)
model.to(device)

(None, None, None, None)
(None,)
(None,)
(None, None, None, None)
(None,)
(None,)
(None, None, None, None)
(None,)
(None,)
(None, None, None, None)
(None,)
(None,)
(None, None, None, None)
(None,)
(None,)
(None, None, None, None)
(None,)
(None,)
(None, None, None, None)
(None,)
(None,)
(None, None, None, None)
(None,)
(None,)
(None, None, None, None)
(None,)
(None,)
(None, None, None, None)
(None,)
(None,)
(None, None, None, None)
(None,)
(None,)
(None, None, None, None)
(None,)
(None,)
(None, None, None, None)
(None,)
(None,)
(None, None, None, None)
(None,)
(None,)
(None, None, None, None)
(None,)
(None,)
(None, None, None, None)
(None,)
(None,)
(None, None, None, None)
(None,)
(None,)
(None, None, None, None)
(None,)
(None,)
(None, None, None, None)
(None,)
(None,)
(None, None, None, None)
(None,)
(None,)
(None, None, None, None)
(None,)
(None,)
(None, None, None, None)
(None,)
(None,)
(None, None, None, None)
(None,)
(None,)
(None, None, None, None)
(None,)
(None,)
(None, None, Non

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [10]:
criterion = nn.CrossEntropyLoss()
# optimizer = optim.Adam(model.parameters(), lr=0.0001,
#                        weight_decay=1e-4)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
#scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[25], gamma=0.1)

In [ ]:
# 1st Training
total_epoch = 100
# 20
print_per_iteration = 5 
# 100
save_path = './model.pth'
train_loss = 0
for epoch in range(total_epoch):  # loop over the dataset multiple times
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        # zero the parameter gradients
        optimizer.zero_grad()
        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        #scheduler.step()
        train_loss += loss.item()

        # print statistics
        if (i+1) % print_per_iteration == 0:    # print every 2000 mini-batches
            print(f'[ep {epoch + 1}][{i + 1:5d}/{len(trainloader):5d}] [Acc] [{(train_loss/(i+1))*100:.3f}%] loss: {loss.item():.3f}')
    torch.save(model.state_dict(), save_path)

In [12]:
# 2nd Training
model = models.resnet50(pretrained=False)

fc_inputs = model.fc.in_features 
model.fc = nn.Sequential(         
    nn.Linear(fc_inputs, num_classes),  
    nn.LogSoftmax(dim=1)
)


model.load_state_dict(torch.load('./model.pth'))
model_arg = model.modules()
for p in model_arg:
    if p._get_name()!='Linear':
        print(p._get_name())
        p.requires_grad_=False
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

model.to(device)

FileNotFoundError: ignored

In [ ]:
total_epoch = 30
# 20
print_per_iteration = 5 
# 100
save_path = './model.pth'
train_loss = 0
for epoch in range(total_epoch):  # loop over the dataset multiple times
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        # zero the parameter gradients
        optimizer.zero_grad()
        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        #scheduler.step()
        train_loss += loss.item()

        # print statistics
        if (i+1) % print_per_iteration == 0:    # print every 2000 mini-batches
            print(f'[ep {epoch + 1}][{i + 1:5d}/{len(trainloader):5d}] [Acc] [{(train_loss/(i+1))*100:.3f}%] loss: {loss.item():.3f}')
    torch.save(model, save_path)

In [15]:
# load trained model
model = torch.load("./model.pth")
model.to(device)

# fixed testing process
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)
        # calculate outputs by running images through the network
        outputs = model(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        #print(predicted)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print(f'Accuracy of the network on the 10000 test images: {100 * correct / total:.2f} %')

Accuracy of the network on the 10000 test images: 58.55 %


Accuracy of the network on the 10000 test images: 60.19 %

In [10]:
# model = models.mobilenet_v3_large()
# torch.save(model, "./model.pth")

# see size of saved model
! du -h model.pth

91M	model.pth
